In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, count, round, when, sum, lit

In [3]:
spark = SparkSession.builder.appName("DE_hw_03").getOrCreate()

1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

In [14]:
users_df = spark.read.csv('./users.csv', header=True)
users_df.show(10)

+-------+-------+---+------------------+
|user_id|   name|age|             email|
+-------+-------+---+------------------+
|      1| User_1| 45| user1@example.com|
|      2| User_2| 48| user2@example.com|
|      3| User_3| 36| user3@example.com|
|      4| User_4| 46| user4@example.com|
|      5| User_5| 29| user5@example.com|
|      6| User_6| 39| user6@example.com|
|      7| User_7| 24| user7@example.com|
|      8| User_8| 44| user8@example.com|
|      9| User_9| 27| user9@example.com|
|     10|User_10| 43|user10@example.com|
+-------+-------+---+------------------+
only showing top 10 rows



In [6]:
products_df = spark.read.csv('./products.csv', header=True)
products_df.show(10)

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
|         6|   Product_6|     Sports|  8.7|
|         7|   Product_7|     Beauty|  8.2|
|         8|   Product_8|     Sports|  1.0|
|         9|   Product_9|     Beauty|  6.0|
|        10|  Product_10|     Sports|  5.4|
+----------+------------+-----------+-----+
only showing top 10 rows



In [7]:
purchases_df = spark.read.csv('./purchases.csv', header=True)
purchases_df.show(10)

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
|          6|     21|        24|2022-01-06|       7|
|          7|     83|        15|2022-01-07|       7|
|          8|     87|        32|2022-01-08|       3|
|          9|     75|        32|2022-01-09|       2|
|         10|     75|        24|2022-01-10|       9|
+-----------+-------+----------+----------+--------+
only showing top 10 rows



In [ ]:
users_df.describe().show()

+-------+------------------+-------+-----------------+-------------------+
|summary|           user_id|   name|              age|              email|
+-------+------------------+-------+-----------------+-------------------+
|  count|               100|     98|               98|                 99|
|   mean|              50.5|   NULL|35.13265306122449|               NULL|
| stddev|29.011491975882016|   NULL|9.168117947710689|               NULL|
|    min|                 1| User_1|               18|user100@example.com|
|    max|                99|User_99|               49|  user9@example.com|
+-------+------------------+-------+-----------------+-------------------+



2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.

In [33]:
users_df = users_df.dropna()
users_df.describe().show()

+-------+------------------+-------+-----------------+-------------------+
|summary|           user_id|   name|              age|              email|
+-------+------------------+-------+-----------------+-------------------+
|  count|                95|     95|               95|                 95|
|   mean|              49.6|   NULL|34.96842105263158|               NULL|
| stddev|28.691462144686874|   NULL|9.165676759472076|               NULL|
|    min|                 1| User_1|               18|user100@example.com|
|    max|                98|User_98|               49|  user9@example.com|
+-------+------------------+-------+-----------------+-------------------+



In [34]:
products_df = products_df.dropna()
products_df.describe().show()

+-------+------------------+------------+--------+------------------+
|summary|        product_id|product_name|category|             price|
+-------+------------------+------------+--------+------------------+
|  count|                47|          47|      47|                47|
|   mean|25.340425531914892|        NULL|    NULL| 5.919148936170212|
| stddev|14.713608896919473|        NULL|    NULL|2.5341698149983642|
|    min|                 1|   Product_1|  Beauty|               1.0|
|    max|                 9|   Product_9|  Sports|               9.7|
+-------+------------------+------------+--------+------------------+



In [35]:
purchases_df = purchases_df.dropna()
purchases_df.describe().show()

+-------+------------------+------------------+------------------+----------+-----------------+
|summary|       purchase_id|           user_id|        product_id|      date|         quantity|
+-------+------------------+------------------+------------------+----------+-----------------+
|  count|               195|               195|               195|       195|              195|
|   mean|100.32820512820513|  49.8974358974359|26.456410256410255|      NULL|4.912820512820513|
| stddev| 57.81150470848114|29.765401342881777|14.540802791118644|      NULL|2.570167819862638|
|    min|                 1|                 1|                 1|2022-01-01|                1|
|    max|                99|                99|                 9|2022-07-19|                9|
+-------+------------------+------------------+------------------+----------+-----------------+



3. Визначте загальну суму покупок за кожною категорією продуктів.

In [36]:
all_df = purchases_df.join(products_df, purchases_df.product_id == products_df.product_id) \
.join(users_df, purchases_df.user_id == users_df.user_id)
all_df.show()

+-----------+-------+----------+----------+--------+----------+------------+-----------+-----+-------+--------+---+-------------------+
|purchase_id|user_id|product_id|      date|quantity|product_id|product_name|   category|price|user_id|    name|age|              email|
+-----------+-------+----------+----------+--------+----------+------------+-----------+-----+-------+--------+---+-------------------+
|          1|     52|         9|2022-01-01|       1|         9|   Product_9|     Beauty|  6.0|     52| User_52| 39| user52@example.com|
|          2|     93|        37|2022-01-02|       8|        37|  Product_37|   Clothing|  6.0|     93| User_93| 25| user93@example.com|
|          3|     15|        33|2022-01-03|       1|        33|  Product_33|       Home|  9.4|     15| User_15| 30| user15@example.com|
|          4|     72|        42|2022-01-04|       9|        42|  Product_42|     Beauty|  9.1|     72| User_72| 39| user72@example.com|
|          6|     21|        24|2022-01-06|     

In [37]:
all_df = all_df.withColumn('total', round(col('quantity') * col('price'), 2))
all_df.show()

+-----------+-------+----------+----------+--------+----------+------------+-----------+-----+-------+--------+---+-------------------+-----+
|purchase_id|user_id|product_id|      date|quantity|product_id|product_name|   category|price|user_id|    name|age|              email|total|
+-----------+-------+----------+----------+--------+----------+------------+-----------+-----+-------+--------+---+-------------------+-----+
|          1|     52|         9|2022-01-01|       1|         9|   Product_9|     Beauty|  6.0|     52| User_52| 39| user52@example.com|  6.0|
|          2|     93|        37|2022-01-02|       8|        37|  Product_37|   Clothing|  6.0|     93| User_93| 25| user93@example.com| 48.0|
|          3|     15|        33|2022-01-03|       1|        33|  Product_33|       Home|  9.4|     15| User_15| 30| user15@example.com|  9.4|
|          4|     72|        42|2022-01-04|       9|        42|  Product_42|     Beauty|  9.1|     72| User_72| 39| user72@example.com| 81.9|
|     

3. Визначте загальну суму покупок за кожною категорією продуктів.

In [43]:
all_df.groupBy('category') \
    .agg(round(sum(col('total')), 2).alias('total_sales')) \
    .orderBy(col('total_sales').desc()) \
    .show()

+-----------+-----------+
|   category|total_sales|
+-----------+-----------+
|     Sports|     1755.5|
|       Home|     1438.9|
|Electronics|     1141.9|
|   Clothing|      696.1|
|     Beauty|      441.7|
+-----------+-----------+



4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

In [45]:
all_df.where("age between 18 and 25") \
    .groupBy('category') \
    .agg(round(sum(col('total')), 2).alias('total_sales')) \
    .orderBy(col('total_sales').desc()) \
    .show()

+-----------+-----------+
|   category|total_sales|
+-----------+-----------+
|       Home|      361.1|
|     Sports|      310.5|
|Electronics|      249.6|
|   Clothing|      245.0|
|     Beauty|       41.4|
+-----------+-----------+



5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.

In [63]:
total_sum = all_df.where("age between 18 and 25") \
    .groupBy('category') \
    .agg(round(sum(col('total')), 2).alias('total_sales')) \
    .agg(sum('total_sales').alias('total_sum')) \
    .first()['total_sum']

all_df.where("age between 18 and 25") \
    .groupBy('category') \
    .agg(round(sum(col('total')), 2).alias('total_sales')) \
    .withColumn('%_of_total', round(col('total_sales') / lit(total_sum) * 100, 2)) \
    .orderBy(col('total_sales').desc()) \
    .show()

+-----------+-----------+----------+
|   category|total_sales|%_of_total|
+-----------+-----------+----------+
|       Home|      361.1|      29.9|
|     Sports|      310.5|     25.71|
|Electronics|      249.6|     20.67|
|   Clothing|      245.0|     20.29|
|     Beauty|       41.4|      3.43|
+-----------+-----------+----------+



6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [64]:
all_df.where("age between 18 and 25") \
    .groupBy('category') \
    .agg(round(sum(col('total')), 2).alias('total_sales')) \
    .withColumn('%_of_total', round(col('total_sales') / lit(total_sum) * 100, 2)) \
    .orderBy(col('total_sales').desc()) \
    .limit(3) \
    .show()

+-----------+-----------+----------+
|   category|total_sales|%_of_total|
+-----------+-----------+----------+
|       Home|      361.1|      29.9|
|     Sports|      310.5|     25.71|
|Electronics|      249.6|     20.67|
+-----------+-----------+----------+



In [65]:
spark.stop()